In [8]:
import pandas as pd
import numpy as np
import random
import math
import time

#### Sampling Task

In [9]:
def minWise(data,k):
    reservoir = []
    sample = []
    for index,d in data.iterrows():
        rnd = random.uniform(0,1)
        # first fill the reservoir
        if len(reservoir) < k:
            reservoir.append(rnd)
            sample.append(d)
        try:
            # Based on random number, replace Netflows with new ones
            idx = next(x[0] for x in enumerate(reservoir) if x[1] > rnd)
            reservoir[idx] = rnd
            sample[idx] = d
        except StopIteration:
            continue
        except ValueError:
            continue
    return pd.DataFrame(sample,columns=data.columns.values,index=range(k))

In [3]:

'''
# preporecessing step
netflw1 = pd.read_csv('capture20110815-2.pcap.netflow.csv')
print(netflw1.head())
#print(netflw1.columns.values)
#netflw1 = netflw1.drop('->',axis=1)
#netflw1 = netflw1.rename(columns={'#Date_flow':'DateFlow','Label(LEGITIMATE:Botnet:Background)':'Label',
#                                  'Src_IP_Addr:Port':'Src','Dst_IP_Addr:Port':'Dst'})

print(len(netflw1))
#netslice = netflw1.loc[1:30,:]
netflw1[['Src_Addr','Src_Port']] = netflw1['Src'].str.split(':',1,expand=True)
netflw1[['Dst_Addr','Dst_Port']] = netflw1['Dst'].str.split(':',1,expand=True)
netflw1.drop(['Src','Dst'], inplace=True, axis=1)
netflw1.to_csv('capture20110815.pcap.netflow.csv')
'''

"\n# preporecessing step\nnetflw1 = pd.read_csv('capture20110815-2.pcap.netflow.csv')\nprint(netflw1.head())\n#print(netflw1.columns.values)\n#netflw1 = netflw1.drop('->',axis=1)\n#netflw1 = netflw1.rename(columns={'#Date_flow':'DateFlow','Label(LEGITIMATE:Botnet:Background)':'Label',\n#                                  'Src_IP_Addr:Port':'Src','Dst_IP_Addr:Port':'Dst'})\n\nprint(len(netflw1))\n#netslice = netflw1.loc[1:30,:]\nnetflw1[['Src_Addr','Src_Port']] = netflw1['Src'].str.split(':',1,expand=True)\nnetflw1[['Dst_Addr','Dst_Port']] = netflw1['Dst'].str.split(':',1,expand=True)\nnetflw1.drop(['Src','Dst'], inplace=True, axis=1)\nnetflw1.to_csv('capture20110815.pcap.netflow.csv')\n"

In [10]:
# netflow file has been pre-processed. src and dst ip addressed have been splited from corresponding port addresses
# read pre-processed file
df = pd.read_csv('capture20110815.pcap.netflow.csv')
#print(df.head())

C:\Users\arica\AppData\Local\conda\conda\envs\my_root\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
# count ip ocurrences to find infected host. the most common ip address has been chosen as infected host
# which indicates the most frequent connections

# drop other connections
df = df[(df['Src_Addr'] == '147.32.84.229') | (df['Dst_Addr'] == '147.32.84.229')]
df = df.dropna()
#print(df.head())

In [12]:
# calculate ip address frequency which are connected with the infected host
ip_count = {}
# count outgoing connections
for ip in df['Dst_Addr']:
    if ip == '147.32.84.229':
        continue
    if ip in ip_count.keys():
        ip_count[ip] += 1
    else:
        ip_count[ip] = 1

# count in coming connections
for ip in df['Src_Addr']:
    if ip == '147.32.84.229':
        continue
    if ip in ip_count.keys():
        ip_count[ip] += 1
    else:
        ip_count[ip] = 1

In [13]:
#for ip in ip_count:
#    print('IP: {}\t Count: {}'.format(ip,ip_count[ip]))

top_IPs = sorted(ip_count, key=ip_count.get, reverse=True)[:10]

In [14]:
# Statistics of distribution
dist = []
sample = []
sample.append(ip_count)

In [15]:
# top 10 IP addresses
print('\nTop 10 IP Adresses with relative frequencies\n')
for ip in top_IPs:
    print('IP: {}\tCount: {}'.format(ip,ip_count[ip]))


Top 10 IP Adresses with relative frequencies

IP: 81.90.162.3	Count: 445
IP: 208.88.186.6	Count: 427
IP: 87.100.20.56	Count: 407
IP: 193.165.178.157	Count: 387
IP: 161.111.235.112	Count: 383
IP: 94.209.77.164	Count: 372
IP: 145.246.240.14	Count: 372
IP: 77.71.211.112	Count: 305
IP: 93.104.50.143	Count: 291
IP: 135.196.109.101	Count: 273


In [16]:
estimation = []
# Sampling process
for k in range(10000,20000,2000):
    print('\nSample Size k: {}'.format(k))
    start = time.time()
    mw10000 = minWise(df,k)
    ellapsed_time = time.strftime("%H:%M:%S", time.gmtime(time.time() - start))
    print('Sample generation time : {}'.format(ellapsed_time))

    ip_count_sample = {}
    # count out going connections
    for ip in mw10000['Dst_Addr']:
        if ip == '147.32.84.229':
            continue
        if ip in ip_count_sample.keys():
            ip_count_sample[ip] += 1
        else:
            ip_count_sample[ip] = 1

    # count in coming connections
    for ip in mw10000['Src_Addr']:
        if ip == '147.32.84.229':
            continue
        if ip in ip_count_sample.keys():
            ip_count_sample[ip] += 1
        else:
            ip_count_sample[ip] = 1
    
    # compute estimated ocurrences
    ip_estimate_sample = {}
    top_IPs_sample = sorted(ip_count_sample, key=ip_count_sample.get, reverse=True)[:10]
    mean_sample = sum(ip_count_sample[ip] for ip in ip_count_sample)/len(ip_count_sample)

    for ip in ip_count_sample:
        ip_estimate_sample[ip] = math.ceil((ip_count_sample[ip]/len(mw10000))*len(df))

    estimation.append(ip_estimate_sample)
    
    # compute approximation error statistics
    apprx_err = []
    for ip in ip_estimate_sample:
        apprx_err.append(ip_estimate_sample[ip] - ip_count[ip])

    print('Approximation error mean: {}\nStandard Deviation: {}'
          .format(np.mean(apprx_err),np.std(apprx_err)))


SAmple Size k: 10000
Sample generation time : 00:00:25
Approximation error mean: 22.469573786646958
Standard Deviation: 24.24951897107075

SAmple Size k: 12000
Sample generation time : 00:00:23
Approximation error mean: 18.609781918272283
Standard Deviation: 21.338285501269336

SAmple Size k: 14000
Sample generation time : 00:00:23
Approximation error mean: 16.5293671845396
Standard Deviation: 19.432050206662566

SAmple Size k: 16000
Sample generation time : 00:00:21
Approximation error mean: 14.505825908156272
Standard Deviation: 17.852869487109633

SAmple Size k: 18000
Sample generation time : 00:00:21
Approximation error mean: 13.009073842302879
Standard Deviation: 16.50238668272219


In [24]:
# Top 10 IP Addresses from one of samples
print('\nTop 10 Sampled IP Adresses from sample size {}\n'.format(k)
top_IPs_sample = sorted(estimation[-1], key=estimation[-1].get, reverse=True)[:10]
for ip in top_IPs_sample:
    print('IP: {}\tExpected Count: {}\tEstimation: {}'.format(ip,ip_count[ip],round(estimation[-1][ip],3)))


Top 10 Sampled IP Adresses from sample size 5278

IP: 87.100.20.56	Expected Count: 407	Estimation: 448
IP: 208.88.186.6	Expected Count: 427	Estimation: 411
IP: 81.90.162.3	Expected Count: 445	Estimation: 411
IP: 82.103.138.49	Expected Count: 38	Estimation: 398
IP: 145.246.240.14	Expected Count: 372	Estimation: 374
IP: 94.144.63.10	Expected Count: 101	Estimation: 361
IP: 86.25.197.54	Expected Count: 240	Estimation: 324
IP: 221.120.9.117	Expected Count: 110	Estimation: 324
IP: 188.4.163.203	Expected Count: 63	Estimation: 311
IP: 94.209.77.164	Expected Count: 372	Estimation: 311


#### Sketching Task

In [18]:
class CountMin:
    
    def __init__(self, w, d):
        self.size = w*d
        self.w = w
        self.hash_count = d
        self.cm_array =  [[0]*w for i in range(d)]
        
    def add(self, string):
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.w
            self.cm_array[seed][result] += 1
        
    def point(self, string):
        min = 1000000000000
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.w
            if self.cm_array[seed][result]<min:
                min = self.cm_array[seed][result]
        return min

In [19]:
stream_ips=df['Src_Addr']

In [20]:
print("The number of distinct IP addresses is: %s" %len(set(stream_ips)))

The number of distinct IP addresses is: 32766


In [21]:

# count the frequency of each IP address
freq = Counter(stream_ips)

# we find the top 10 frequent adrress by sorting them
sorted_el = sorted(freq.items(), key=operator.itemgetter(1), reverse = True)

# Print the 10 most frequent IP addresses
for i in range(10):
    print (sorted_el[i])

('147.32.84.229', 76790)
('81.90.162.3', 225)
('208.88.186.6', 214)
('87.100.20.56', 201)
('193.165.178.157', 195)
('161.111.235.112', 193)
('94.209.77.164', 188)
('145.246.240.14', 187)
('77.71.211.112', 152)
('93.104.50.143', 145)


In [2]:
#need visual studio C++ 14.0 to install it
!pip install mmh3 

Exception:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\commands\install.py", line 317, in run
    prefix=options.prefix_path,
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\req\req_set.py", line 742, in install
    **kwargs
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\req\req_install.py", line 831, in install
    self.move_wheel_files(self.source_dir, root=root, prefix=prefix)
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\req\req_install.py", line 1032, in move_wheel_files
    isolated=self.isolated,
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\wheel.py", line 346, in move_wheel_files
    clobber(source, lib_dir, True)
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\wheel.py", line 324, in clobber
    shutil.copyfile(srcfile, destfile)
  File "C:\Program File

In [4]:
#using mmh3 hash function
import operator
from collections import Counter
from bitarray import bitarray
import mmh3
import math

In [22]:
e = 2.718281828

#initialize ε and δ
for epsilon in [0.0001, 0.001, 0.01]:
    for delta in [0.0001, 0.001, 0.01]:
        # derive w and d
        w = round(e/epsilon)
        d = round(math.log(1/delta))

        # create count min
        cm = CountMin(w, d)

        # streaming and adding to matrix
        for ip in stream_ips:
                cm.add(ip)
                
        # find frequency and store it to cm_res
        cm_res = {}
        for ip in stream_ips:
            cm_res[ip] = cm.point(ip)

        # Sort them according to their value to find the 10 most frequent ones
        sorted_cm = sorted(cm_res.items(), key=operator.itemgetter(1), reverse = True)

        diff = 0
        for i in range(10):
            diff+= abs(sorted_cm[i][1] - sorted_el[i][1])

        print("ε = %s | δ = %s | w = %s | d = %s " %(epsilon, delta, w, d))
        print("The total difference between the frequency of the 10 most frequent results of count min and the ground truth: \n%s \n"%diff)

ε = 0.0001 | δ = 0.0001 | w = 27183 | d = 9 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
0 

ε = 0.0001 | δ = 0.001 | w = 27183 | d = 7 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
0 

ε = 0.0001 | δ = 0.01 | w = 27183 | d = 5 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
0 

ε = 0.001 | δ = 0.0001 | w = 2718 | d = 9 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
102 

ε = 0.001 | δ = 0.001 | w = 2718 | d = 7 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
107 

ε = 0.001 | δ = 0.01 | w = 2718 | d = 5 
The total difference between the frequency of the 10 most frequent results of count min and the ground truth: 
118 

ε = 0.01 | δ = 0.0001 | w = 272 | d = 9 
The t